In [1]:
import correct_lidar
import open3d as o3d
from pyntcloud import PyntCloud
# transform 
import numpy as np
import matplotlib.pyplot as plt
# from open3d import JVisualizer
import glob
import rasterio as rio

In [2]:
files = [
    "/Users/elischwat/Downloads/lidar_20230510/2023-05-10_00-00-13.lvx"
]
output_dir = "/Users/elischwat/Downloads/lidar_20230510_pcd"
LIDAR = "l2"

In [26]:
for file in files:
    points = correct_lidar.load_first_file(LIDAR, file_search=file)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points.T)
    o3d.io.write_point_cloud(
        filename=os.path.join(output_dir, os.path.basename(files).replace('.lvx', '.ply')),
        pointcloud=pcd,
    )

In [3]:
import glob
files =sorted(glob.glob("/Users/elischwat/Downloads/lidar_20230510/*.lvx"))
files = [f for f in files if (f.split('/')[-1].split('-')[3]) in ['00', '30']]
len(files), files[:5]

(48,
 ['/Users/elischwat/Downloads/lidar_20230510/2023-05-10_00-00-13.lvx',
  '/Users/elischwat/Downloads/lidar_20230510/2023-05-10_00-30-14.lvx',
  '/Users/elischwat/Downloads/lidar_20230510/2023-05-10_01-00-14.lvx',
  '/Users/elischwat/Downloads/lidar_20230510/2023-05-10_01-30-14.lvx',
  '/Users/elischwat/Downloads/lidar_20230510/2023-05-10_02-00-14.lvx'])

In [4]:
output_dir = "/Users/elischwat/Downloads/lidar_20230510_pcd"
LIDAR = "l2"

In [8]:
for file in files:
    points = correct_lidar.load_first_file(LIDAR, file_search=file)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points.T)
    o3d.io.write_point_cloud(
        filename=os.path.join(output_dir, os.path.basename(file).replace('.lvx', '.ply')),
        pointcloud=pcd,
    )

In [21]:


o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])



In [ ]:
# get all files in the data/l2/snow_off directory
files = glob.glob("data/l2/snow_on/*.lvx")
lidar = "l2"
for file in files:
    points = correct_lidar.load_first_file(lidar, file_search=file)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points.T)
    o3d.io.write_point_cloud(filename=f"./data/l2/snow_on/{lidar}_2022-10-23_01-00.pcd", pointcloud=pcd)

In [15]:
points

array([[  9.42793025,   9.29583167,   9.18963136, ...,  13.83339094,
         13.87095186,  13.90095646],
       [-64.24323324, -61.53314072, -59.20549265, ...,   5.74292405,
          5.79244582,   5.85095531],
       [-15.75949755, -15.48733126, -15.27538552, ..., -10.6750266 ,
        -10.65525865, -10.62589263]])

In [ ]:
points = correct_lidar.read_file(filename=f"./data/snow_off/{lidar}_2022-10-23_*.lvx")


In [16]:
points[:,:3].T

array([[  9.42793025, -64.24323324, -15.75949755],
       [  9.29583167, -61.53314072, -15.48733126],
       [  9.18963136, -59.20549265, -15.27538552]])

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[:,:3])
o3d.io.write_point_cloud(filename=f"./data/snow_off/{lidar}_test.pcd", pointcloud=pcd)

In [ ]:
import rioxarray as rxr

# open the raster
raster = rxr.open_rasterio("data/raster.tif", masked=True).squeeze()

# print the number of nans
print(f"Number of NaNs: {np.isnan(raster).sum()}")
# print the number of non-nans
print(f"Number of non-NaNs: {np.count_nonzero(~np.isnan(raster))}")
raster.plot.imshow(
    #increase point size
    cmap="viridis",
    size=10,
    aspect=1.5,
    robust=True,
)

In [ ]:
def correct_data(points, azimuth, xoffset, yoffset, zoffset, roll=0, elevation_adjustment=0,
                 global_adjustment=True, background_correction=None, correct_l5=False):
    """
    Apply corrections to raw lidar data for known lidar positions/orientations and to correct background light effects.
    Optionally do not reposition points into a common reference across lidars.
    Background light correction only applied if calibration targets are specified.
    """

    # first take out 30deg downward pointing mount for all lidars
    if elevation_adjustment==0:
        pr = rotate_data(v=points[:,:3].T, axis=[0, 0, 1], theta=np.deg2rad(-30.0), rotation=None)   # 31 deg
    else:
        pr = rotate_data(v=points[:,:3].T, axis=[0, 0, 1], theta=np.deg2rad(-30.0+elevation_adjustment), rotation=None)   # 31 deg

    # Then account for mounting on a vertical mount
    if roll==0:
        pr = rotate_data(pr, axis=[1,0,0],theta=np.pi/2)
    else:
        pr = rotate_data(pr, axis=[1,0,0],theta=np.pi/2 + np.deg2rad(roll))

    if correct_l5:
        pr = update_l5(pr)

    if background_correction is not None:
        pr = correct_points_background(pr, background_correction)


    if global_adjustment:
        # Then rotate to a common map orientation
        pr = rotate_data(pr, axis=[0,0,1],theta=np.deg2rad(azimuth))

        # shift location based on lidar mount points
        pr[0,:] += (xoffset - center_point[0])
        pr[1,:] += (yoffset - center_point[1])
        pr[2,:] += zoffset

        # Then rotate to a North map orientation
        pr = rotate_data(pr, axis=[0,0,1],theta=np.deg2rad(global_rotation))


    return pr